In [15]:
import pandas as pd
import numpy as np
from datetime import datetime 
import os
import transcation as tran
import tushare as ts

In [20]:
path = 'C:/Users/Administrator/Desktop/Work/data/CTA_algo/backtesting/result0_0/'
dfs = []
for f in os.listdir(path):
    if f.find('orders.csv') == -1:
        continue
    df = pd.read_csv(os.path.join(path, f), index_col=0, parse_dates=True, 
                     header=None, names=['order_id', 'open_close', 'direction', 'status', 'price', 'lots'])
    df['instrument'] = f[:f.find('_')]
    dfs.append(df)
df = pd.concat(dfs)
df = df[df.status == 1]
df.sort_index(inplace=True)
df.index.name = 'datetime'
df.reset_index(inplace=True)
dfday = pd.read_csv('tradeDate.csv')
dfday.columns = ['zzz','calendarDate']
dfday.drop('zzz',axis=1,inplace=True)
dates = list(dfday.calendarDate)
def business_date(t):
    if t.hour > 15: # afternoon
        return dates[dates.index(t.strftime('%Y-%m-%d')) + 1]
    else:
        return t.strftime('%Y-%m-%d')
    
df['real_trade_date'] = df.datetime.apply(business_date)
df.set_index('datetime', inplace=True)
df.open_close = df.open_close.apply(lambda x : 0 if x == 'O' else 1)
df.direction = df.direction.apply(lambda x : 0 if x == 'B' else 1)
df
open_lots = df[df.open_close == 0].groupby('real_trade_date')['lots'].sum()
close_lots = df[df.open_close == 1].groupby('real_trade_date')['lots'].sum()
pd.DataFrame([open_lots, close_lots]).T
tran.daily_position_from_trancation(df)
 

real_trade_date  instrument  direction_for_position
2016-12-05       cf705       1                           1
                 cs1705      1                           5
                 m1705       0                           0
                             1                           0
                 p1705       0                           0
                 rm705       0                           0
                             1                           0
                 ta705       0                           0
                 y1705       0                           0
2016-12-06       a1705       0                           0
                 cf705       1                           0
                 cs1705      1                           0
                 m1705       0                           0
                 p1705       0                           0
                 rb1705      1                           0
                 rm705       0                           0
    

In [14]:
df = pd.read_hdf('C:/Users/Administrator/Desktop/Work/data/CTAtrade_account/trade_detail_007738030022.h5')
#datetime = pd.to_datetime(df.trade_time)
dt = datetime.strptime('20:59:59', "%H:%M:%S")
trade_date_list = df.real_trade_date.unique().tolist()
def business_date(row):
    t = datetime.strptime(row.trade_time, "%H:%M:%S")
    if t.hour > 15: # afternoon
        return trade_date_list[trade_date_list.index(row.real_trade_date) - 1]
    else:
        return row.real_trade_date
df['business_date'] = df.apply(business_date, axis=1)
df.set_index(pd.to_datetime(df.business_date + ' ' + df.trade_time), inplace=True)
df.sort_index(inplace=True)
df =  df[['instrument', 'direction', 'order_price', 'lots', 'open_close', 'real_trade_date']]
tran.daily_position_from_trancation(df)
#df.groupby(['real_trade_date','instrument','open_close'])['lots'].sum()
#df

real_trade_date  instrument  direction_for_position
2016-12-05       CF705       1                          1
                 CS1705      1                          5
                 M1705       0                          0
                             1                          0
                 P1705       0                          0
                 RM705       0                          0
                             1                          0
                 TA705       0                          0
                 Y1705       0                          0
2016-12-06       A1705       0                          0
                 CF705       1                          0
                 CS1705      1                          0
                 M1705       0                          0
                 P1705       0                          0
                 RB1705      1                          0
                 RM705       0                          0
                 RU1

In [58]:
#如果是开仓则数量为正数,平仓则数量为负数
#df['position_effect'] = df.apply(lambda x : x.lots if x.open_close == 0 else -x.lots, axis=1)
# 将平仓订单的方向改为相反方向,用来计算持仓
#df['direction_for_position'] = df.apply(reverse_direction_if_close, axis=1)
#df.reset_index(inplace=True)

#instruments = df.instrument.unique().tolist()
#series_list = []
#for instrument in instruments:
#    for direction in range(2):
#        series = df[(df.instrument == instrument) & (df.direction_for_position == direction)]['position_effect'].cumsum()
#        series_list.append(series)
#df['position_cumsum'] = pd.concat(series_list).sort_index()

#df[(df.instrument == 'M1705') & (df.direction_for_position == 0)].position_effect.cumsum()
#df.groupby(['real_trade_date','instrument', 'direction_for_position' ])['position_cumsum'].last()

In [59]:
#
#instrument_df.index.__class__
#records  = instrument_df.to_records(index=True, convert_datetime64=False)
#row = records.astype(np.dtype([('trade_datetime', '<M8[ns]'), ('order_no', 'S40')]))
#row
#new_descr = []
#existing_descr = records.dtype.descr
#
#for idx,d in enumerate(existing_descr):
#    if existing_descr[idx][1] == '|O' and dest_dtype[idx].char == 'S' and convert_strings:
#        # records dtype is something like |O8 and dest dt is a string
#        new_descr.append((existing_descr[idx][0], dest_dtype[idx]))
#    elif idx == 0:
#        # Make sure timestamp is in milliseconds
#        new_descr.append((existing_descr[idx][0], '<M8[ms]'))
#    else:
#        new_descr.append(existing_descr[idx])
#
## recast to the new type
#rows = records.astype(numpy.dtype(new_descr))
#rows


#instrument_df.order_no = instrument_df.order_no.str.decode('utf-8')
#records = instrument_df.to_records(index=True,convert_datetime64=False)
#records.dtype
#instrument_df.order_no.astype(dtype='S10')
#pd.read_hdf('d:/trade_detail_007738030022.h5')